In [1]:
import pickle
import pandas as pd
from sklearn.linear_model import LogisticRegression

In [2]:
X_test_1 = pd.read_feather('../input/amex-imputed-and-1hot-encoded/X_test_1.ftr')
X_test_1 = X_test_1.set_index('customer_ID')

In [3]:
X_test_2 = pd.read_feather('../input/amex-imputed-and-1hot-encoded/X_test_2.ftr')
X_test_2 = X_test_2.set_index('customer_ID')

In [4]:
model = pickle.load(open('/kaggle/input/test-dtc-model/logistic_regression_model.sav', 'rb'))
preds_1 = pd.DataFrame(model.predict_proba(X_test_1)[:, 1], index=X_test_1.index, columns=['prediction'])
preds_2 = pd.DataFrame(model.predict_proba(X_test_2)[:, 1], index=X_test_2.index, columns=['prediction'])

In [5]:
submission = pd.concat([preds_1, preds_2])

# predictions only need to be for each customer
submission = submission.groupby('customer_ID').agg(['last'])
submission.columns = submission.columns.droplevel(1)

# predictions need to be doubles 
submission['prediction'] = submission['prediction'].astype('double')

# index needs to be removed from submission csv
submission = submission.reset_index()
submission.to_csv('submission.csv', index=False)